<a href="https://colab.research.google.com/github/nicchic/NLP/blob/main/mental_gpa2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [35]:
model_name = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [107]:
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Nucleus Search

In [43]:
pip install datasets

In [108]:
from datasets import load_dataset

ds = load_dataset("Amod/mental_health_counseling_conversations")

In [109]:
def tokenize_function(examples):
    context_tokens = tokenizer(examples['Context'], padding="max_length", truncation=True, max_length=128)
    response_tokens = tokenizer(examples['Response'], padding="max_length", truncation=True, max_length=128)

    return {
        "context_input_ids": context_tokens["input_ids"],
        "context_attention_mask": context_tokens["attention_mask"],
        "response_input_ids": response_tokens["input_ids"],
        "response_attention_mask": response_tokens["attention_mask"]
    }

In [110]:
tokenized_ds = ds.map(tokenize_function, batched=True)

In [111]:
attention_mask = torch.tensor(tokenized_ds['train']['context_attention_mask'][0]).unsqueeze(0).to(device)


In [112]:
output = model.generate(input_ids,
                        max_length=max_new_tokens,
                        attention_mask=attention_mask,
                        pad_token_id=tokenizer.eos_token_id,
                        num_beams=5,
                        do_sample= False,
                        no_repeat_ngram_size =4
                        )

print(tokenizer.decode(output[0]))

NameError: name 'max_new_tokens' is not defined

In [113]:
input_ids = torch.tensor(tokenized_ds['train']['context_input_ids'][0]).unsqueeze(0).to(device)

output = model.generate(input_ids,
                        attention_mask=attention_mask,
                        pad_token_id=tokenizer.eos_token_id,
                        max_length= 200,
                        do_sample=True,
                        top_p=0.9,
                        )

print(tokenizer.decode(output[0], skip_special_tokens=True))


I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?This is my own view of how I've felt, and I can change it if I'm willing to accept it. I think it could be a simple change that would help me get through this. But it's hard and it makes me feel worthless.I know I shouldn't be in my current state, but I'm going through some things with my feelings


In [114]:
# Greedy decoding
output = model.generate(input_ids,
                        attention_mask=attention_mask,
                        max_new_tokens=128,
                        do_sample=False,
                        pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(output[0], skip_special_tokens=True))

I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?The only way to change my feeling of being worthless is to change my life.
   I'm not a person who's going to be able to change my life. I'm not a person who's going to be able to change my life. I'm not a person who's going to be able to change my life. I'm not a person who's going to be able to change my life. I'm not a person who's going to be able to change my life. I'm not a person who's going to be able to change my life. I'm not a person who's going to be able to change
